In [10]:
from mbientlab.warble import *
from mbientlab.metawear import *
from threading import Event
import time

In [2]:
e = Event()
address = None
def device_discover_task(result):
    global address
    if (result.has_service_uuid(MetaWear.GATT_SERVICE)):
        # grab the first discovered metawear device
        address = result.mac
        e.set()

BleScanner.set_handler(device_discover_task)
BleScanner.start()
e.wait()

BleScanner.stop()

In [4]:
print(address)
device = MetaWear(address)
device.connect()

D4:06:BD:85:8F:23


In [5]:
pattern= LedPattern(repeat_count= Const.LED_REPEAT_INDEFINITELY)
libmetawear.mbl_mw_led_load_preset_pattern(byref(pattern), LedPreset.BLINK)
libmetawear.mbl_mw_led_write_pattern(device.board, byref(pattern), LedColor.GREEN)
libmetawear.mbl_mw_led_play(device.board)

In [6]:
libmetawear.mbl_mw_led_stop_and_clear(device.board)

In [11]:
e = Event()
d = device
# Get the accelerometer data signal
signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
# Create a logger
logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)
# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)

print("Logging data for 5s")
time.sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))

# Stop logger
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

Logging data for 5s
{epoch: 1713865285289, value: {x : 0.035, y : -0.997, z : 0.003}}
{epoch: 1713865285299, value: {x : 0.032, y : -0.998, z : 0.006}}
{epoch: 1713865285309, value: {x : 0.027, y : -0.998, z : -0.002}}
{epoch: 1713865285320, value: {x : 0.025, y : -1.000, z : -0.007}}
{epoch: 1713865285330, value: {x : 0.023, y : -0.997, z : -0.010}}
{epoch: 1713865285340, value: {x : 0.025, y : -0.994, z : -0.009}}
{epoch: 1713865285350, value: {x : 0.024, y : -0.995, z : -0.011}}
{epoch: 1713865285359, value: {x : 0.028, y : -0.997, z : -0.010}}
{epoch: 1713865285369, value: {x : 0.032, y : -0.999, z : -0.006}}
{epoch: 1713865285380, value: {x : 0.031, y : -1.000, z : -0.006}}
{epoch: 1713865285390, value: {x : 0.033, y : -1.000, z : -0.003}}
{epoch: 1713865285400, value: {x : 0.030, y : -0.998, z : 0.002}}
{epoch: 1713865285410, value: {x : 0.028, y : -0.998, z : 0.004}}
{epoch: 1713865285421, value: {x : 0.028, y : -1.000, z : 0.002}}
{epoch: 1713865285431, value: {x : 0.029, y : -

True